In [ ]:
import socket
import threading
import uuid
import json
import hashlib
import time
import uuid

In [ ]:
### Max 5 connections ==> ändern?
class Card:
  def __init__(self, title, author, content):
    self.id = uuid.uuid4()
    self.title = title
    self.author = author
    self.content = content
    self.comments = {}
    self.votes = 0

  def get_id(self):
    return self.id

  def get_title(self):
    return self.title

  def get_author(self):
    return self.author

  def get_content(self):
    return self.content

  def set_title(self, title):
    self.title = title

  def set_author(self, author):
    self.author = author

  def set_content(self, content):
    self.content = content

  def add_comment(self, comment):
    self.comments[comment.id] = comment

  def remove_comment(self, comment_id, comment_author):
    if comment_id in self.comments:
      if self.comments[comment_id].author == comment_author:
        del self.comments[comment_id]

  def get_all_comments(self):
    return self.comments.values().tolist()

  def upvote(self):
    self.votes += 1

  def downvote(self):
    if self.votes > 0:
      self.votes -= 1

In [ ]:
class Comment:
  def __init__(self, author, content):
    self.id = uuid.uuid4()
    self.author = author
    self.content = content

In [ ]:
class Board:
  def __init__(self):

    self.card_references = {}
    self.not_allowed_to_write = []

  def add_card_reference(self, card_id, peer_id, peer_host, peer_port):
    if peer_id not in self.card_references:
      self.card_references[card_id] = (peer_host, peer_port)

  def not_allowed_to_write(self, user_id):
    self.not_allowed_to_write.append(user_id)

In [ ]:
class Peer_node:
  def __init__(self, host, port):
    self.host=host
    self.port=port

    combined_host_port = f"{host}:{port}"
    self.node_id=hashlib.sha256(combined_host_port.encode()).hexdigest() ## Id of node is hash of ip and port

    self.peers = {}
    self.data_store = {}
    self.server_socket = None
    self.running = False

    print(f"Node {self.node_id} initialized at {self.host}:{self.port}")

  ### Node Properties
  def get_id(self):
    return self.node_id

  def get_host(self):
    return self.host

  def get_port(self):
    return self.port

  def get_peers(self):
    return self.peers

  ### Connection to and communication with other peers

  def start(self):
        """Starts the node, listening for incoming connections."""
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        self.server_socket.bind((self.host, self.port))
        self.server_socket.listen(5) # Max 5 queued connections
        self.running = True
        print(f"Node {self.node_id} listening on {self.host}:{self.port}")

        # Start a new thread to continuously accept connections
        threading.Thread(target=self._accept_connections, daemon=True).start()

  def _accept_connections(self):
        """Internal method to accept incoming connections."""
        while self.running:
            try:
                conn, addr = self.server_socket.accept()
                print(f"Incoming connection from {addr}")
                # Handle connection in a new thread
                threading.Thread(target=self._handle_client_connection, args=(conn, addr), daemon=True).start()
            except Exception as e:
                if self.running:
                    print(f"Error accepting connection: {e}")
                break

  def _handle_client_connection(self, conn, addr):
        """Handles a single client connection (a peer)."""
        try:
            # First, receive the peer's node_id and address
            initial_data = conn.recv(1024).decode('utf-8')
            peer_info = json.loads(initial_data)
            peer_id = peer_info.get('node_id')
            peer_host = peer_info.get('host')
            peer_port = peer_info.get('port')

            if peer_id and peer_host and peer_port:
                print(f"Received peer info from {peer_id} at {peer_host}:{peer_port}")
                self.add_peer(peer_id, conn, (peer_host, peer_port))
                #self.send_message_to_peer(peer_id, "ACK: Connected!") # Acknowledge connection
                # Send our own node info to the peer
                my_info = json.dumps({'node_id': self.node_id, 'host': self.host, 'port': self.port})
                conn.sendall(my_info.encode('utf-8'))

                while True:
                    data = conn.recv(4096)
                    if not data:
                        print(f"Peer {peer_id} disconnected.")
                        #self.remove_peer(peer_id)                                #####?????
                        break
                    message = json.loads(data.decode('utf-8'))
                    self.process_message(peer_id, message)
            else:
                print(f"Invalid initial data from {addr}. Closing connection.")
                conn.close()

        except Exception as e:
            print(f"Error handling client connection from {addr}: {e}")
            conn.close()
            # If peer was added, remove it
            for peer_id, (socket_obj, _) in list(self.peers.items()):
                if socket_obj == conn:
                    self.remove_peer(peer_id)
                    break


  def connect_to_peer(self, peer_host, peer_port):
        """Connects to another peer node."""
        if (peer_host, peer_port) == (self.host, self.port):
            print("Cannot connect to self.")
            return False

        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.connect((peer_host, peer_port))

            # Send our own node info to the peer
            my_info = json.dumps({'node_id': self.node_id, 'host': self.host, 'port': self.port})
            s.sendall(my_info.encode('utf-8'))

            # Wait for peer's ack and initial info
            response = s.recv(1024).decode('utf-8')
            response_data = json.loads(response)
            peer_id = response_data.get('node_id') # Assume peer also sends its ID back

            if peer_id and peer_id not in self.peers:
                print(f"Successfully connected to peer {peer_id} at {peer_host}:{peer_port}")
                self.add_peer(peer_id, s, (peer_host, peer_port))

                # Start a thread to listen for messages from this peer
                threading.Thread(target=self._listen_to_peer, args=(peer_id, s), daemon=True).start()
                return True
            else:
                print(f"Already connected to or invalid response from {peer_host}:{peer_port}")
                s.close()
                return False
        except Exception as e:
            print(f"Could not connect to peer {peer_host}:{peer_port}: {e}")
            return False

  def _listen_to_peer(self, peer_id, sock):
        """Listens for messages from a specific connected peer."""
        try:
            while True:
                data = sock.recv(4096)
                if not data:
                    print(f"Peer {peer_id} disconnected during listen.")
                    #self.remove_peer(peer_id)                                  #### ???
                    break
                message = json.loads(data.decode('utf-8'))
                self.process_message(peer_id, message)
        except Exception as e:
            print(f"Error listening to peer {peer_id}: {e}")
            self.remove_peer(peer_id)

  def add_peer(self, peer_id, socket_obj, address_tuple):
        """Adds a new peer to the node's peer list."""
        if peer_id not in self.peers:
            self.peers[peer_id] = (socket_obj, address_tuple)
            print(f"Added peer: {peer_id} from {address_tuple}")
        else:
            print(f"Peer {peer_id} already in peer list.")

  def remove_peer(self, peer_id):
        """Removes a peer from the node's peer list."""
        if peer_id in self.peers:
            socket_obj, _ = self.peers[peer_id]
            try:
                socket_obj.close()
            except Exception as e:
                print(f"Error closing socket for peer {peer_id}: {e}")
            del self.peers[peer_id]
            print(f"Removed peer: {peer_id}")

  def send_message_to_peer(self, peer_id, message_type, content=None):
        """Sends a message to a specific peer."""
        if peer_id in self.peers:
            sock, _ = self.peers[peer_id]
            try:
                full_message = {'sender_id': self.node_id, 'type': message_type, 'content': content}
                sock.sendall(json.dumps(full_message).encode('utf-8'))
                print(f"Sent '{message_type}' to {peer_id}")
            except Exception as e:
                print(f"Error sending message to {peer_id}: {e}")
                self.remove_peer(peer_id)
        else:
            print(f"Peer {peer_id} not found in peer list.")

  def broadcast_message(self, message_type, content=None):
        """Sends a message to all connected peers."""
        print(f"Broadcasting message: Type={message_type}, Content={content}")
        for peer_id in list(self.peers.keys()): # Use list to avoid issues if peers are removed during iteration
            self.send_message_to_peer(peer_id, message_type, content)

  def process_message(self, sender_id, message):
        """Processes an incoming message from a peer."""
        message_type = message.get('type')
        content = message.get('content')
        print(f"Node {self.node_id} received message from {sender_id}: Type='{message_type}', Content='{content}'")

        if message_type == 'text_message':
            print(f"  > Text message: {content}")
        elif message_type == 'data_request':
            key = content.get('key')
            if key in self.data_store:
                self.send_message_to_peer(sender_id, 'data_response', {'key': key, 'value': self.data_store[key]})
            else:
                self.send_message_to_peer(sender_id, 'data_response', {'key': key, 'value': None, 'error': 'not_found'})
        elif message_type == 'data_update':
            key = content.get('key')
            value = content.get('value')
            self.data_store[key] = value
            print(f"  > Stored data: {key} = {value}")
            # Optionally, rebroadcast the update to other peers
            # self.broadcast_message('data_update', content)
        elif message_type == 'get_peers':
            # Send back a list of this node's known peers (host, port)
            peer_addresses = [peer_info[1] for peer_info in self.peers.values()]
            self.send_message_to_peer(sender_id, 'peer_list', {'peers': peer_addresses})
        elif message_type == 'peer_list':
            # Add newly discovered peers to our list
            new_peers = content.get('peers', [])
            for peer_host, peer_port in new_peers:
                if (peer_host, peer_port) != (self.host, self.port) and (peer_host, peer_port) not in [p[1] for p in self.peers.values()]:
                    print(f"Attempting to connect to new peer: {peer_host}:{peer_port}")
                    self.connect_to_peer(peer_host, peer_port)

  def stop(self):
        """Stops the node and closes all connections."""
        self.running = False
        if self.server_socket:
            print(f"Closing server socket for Node {self.node_id}.")
            self.server_socket.close()
        for peer_id in list(self.peers.keys()):
            self.remove_peer(peer_id)
        print(f"Node {self.node_id} stopped.")

  ### Data
  def add_data(self,data):
    self.data_store[data.get_id()]= data

  def get_data(self, data_id):
    if data_id not in self.data_store:
      return None
    return self.data_store[data_id]

  def remove_data(self,data_id):
    del self.data_store[data_id]

  ### Board
  def create_board(self):
    board = Board()
    self = Super_Peer(self, self.host, self.port, board)   #### ????


In [ ]:
class Super_Peer(Peer_node):
  def __init__(self, host, port, board):
    super().__init__(host, port)
    self.board = board

  def delete_board():
    del self.board
    ### Covertieren zum Normalen Peer Node ????

In [ ]:
### Base network
node1 = Peer_node("127.0.0.1", 8001)
node2 = Peer_node("127.0.0.1", 8002)
node3 = Peer_node("127.0.0.1", 8003)

node1.start()
node2.start()
node3.start()

time.sleep(1) # Give servers a moment to start

# Form mesh-like peer network
node1.connect_to_peer("127.0.0.1", 8002)
time.sleep(1)
node2.connect_to_peer("127.0.0.1", 8001)
time.sleep(1)
node3.connect_to_peer("127.0.0.1", 8003)

Node 8eb6c0be28076f317f5a9a1d6967fb3961bd7fb85cf0abb9208ec5a77f758592 initialized at 127.0.0.1:8010
Node 575dcebc38bf0b40bd7d9d1712e723123f6b5900fab82b53c0614dce3335fd20 initialized at 127.0.0.1:8011
Node cb50bbd317ea1495cc6da4e0dc4bef2dcc617baed2e3e7f20d1874ba0283ed0b initialized at 127.0.0.1:8012
Node 8eb6c0be28076f317f5a9a1d6967fb3961bd7fb85cf0abb9208ec5a77f758592 listening on 127.0.0.1:8010
Node 575dcebc38bf0b40bd7d9d1712e723123f6b5900fab82b53c0614dce3335fd20 listening on 127.0.0.1:8011
Node cb50bbd317ea1495cc6da4e0dc4bef2dcc617baed2e3e7f20d1874ba0283ed0b listening on 127.0.0.1:8012


In [ ]:
time.sleep(2) # Give connections a moment to establish

print("\n--- Sending Messages ---")
node1.send_message_to_peer(node2.node_id, 'text_message', 'Hello from Node 1 to Node 2!')
time.sleep(0.5)
node2.broadcast_message('text_message', 'Greetings from Node 2 to everyone!')
time.sleep(0.5)

print("\n--- Data Operations ---")
node1.send_message_to_peer(node2.node_id, 'data_update', {'key': 'my_data', 'value': 'secret_value'})
time.sleep(0.5)
node3.send_message_to_peer(node1.node_id, 'data_request', {'key': 'my_data'})
time.sleep(0.5)
node3.send_message_to_peer(node2.node_id, 'data_request', {'key': 'non_existent_key'})
time.sleep(0.5)

print("\n--- Peer Discovery Example ---")
# Node 1 asks Node 2 for its peers
node1.send_message_to_peer(node2.node_id, 'get_peers')
time.sleep(1) # Give time for peer list to be sent back and processed

Incoming connection from ('127.0.0.1', 47242)
Received peer info from 8eb6c0be28076f317f5a9a1d6967fb3961bd7fb85cf0abb9208ec5a77f758592 at 127.0.0.1:8010
Added peer: 8eb6c0be28076f317f5a9a1d6967fb3961bd7fb85cf0abb9208ec5a77f758592 from ('127.0.0.1', 8010)
Successfully connected to peer 575dcebc38bf0b40bd7d9d1712e723123f6b5900fab82b53c0614dce3335fd20 at 127.0.0.1:8011
Added peer: 575dcebc38bf0b40bd7d9d1712e723123f6b5900fab82b53c0614dce3335fd20 from ('127.0.0.1', 8011)
Incoming connection from ('127.0.0.1', 57450)
Received peer info from 575dcebc38bf0b40bd7d9d1712e723123f6b5900fab82b53c0614dce3335fd20 at 127.0.0.1:8011
Added peer: 575dcebc38bf0b40bd7d9d1712e723123f6b5900fab82b53c0614dce3335fd20 from ('127.0.0.1', 8011)
Successfully connected to peer cb50bbd317ea1495cc6da4e0dc4bef2dcc617baed2e3e7f20d1874ba0283ed0b at 127.0.0.1:8012
Added peer: cb50bbd317ea1495cc6da4e0dc4bef2dcc617baed2e3e7f20d1874ba0283ed0b from ('127.0.0.1', 8012)
Incoming connection from ('127.0.0.1', 52416)
Received pee

In [ ]:
print(node1.get_peers())
# print(node2.get_peers())
# print(node3.get_peers()

{'fbd7416735a39a2abca7227cfec89912a777fa71e328bea87bc1a95425ca9aaf': (<socket.socket fd=73, family=2, type=1, proto=0, laddr=('127.0.0.1', 42644), raddr=('127.0.0.1', 8008)>, ('127.0.0.1', 8008)), '575dcebc38bf0b40bd7d9d1712e723123f6b5900fab82b53c0614dce3335fd20': (<socket.socket fd=74, family=2, type=1, proto=0, laddr=('127.0.0.1', 47242), raddr=('127.0.0.1', 8011)>, ('127.0.0.1', 8011)), 'cb50bbd317ea1495cc6da4e0dc4bef2dcc617baed2e3e7f20d1874ba0283ed0b': (<socket.socket fd=54, family=2, type=1, proto=0, laddr=('127.0.0.1', 8010), raddr=('127.0.0.1', 52416)>, ('127.0.0.1', 8012))}


In [ ]:
data1 = Data( "Title1", "Author1", "Content1")


node1.add_data(data1)

getdata1 = node1.get_data(data1.get_id())

print(getdata1.get_title())
print(getdata1.get_author())
print(getdata1.get_content())

node1.remove_data(data1.get_id())

node1.get_data(data1.get_id())

print(getdata1.get_title())
print(getdata1.get_author())
print(getdata1.get_content())

Title1
Author1
Content1


KeyError: UUID('e4654633-912e-46e5-8f99-d04518c6e619')

In [ ]:
print("Stopping nodes...")
node1.stop()
node2.stop()
node3.stop()

Stopping nodes...
Closing server socket for Node 354f8a05c517336e1929050f3db9e20517757ea252984a2793af897142938c1f.
Removed peer: fbd7416735a39a2abca7227cfec89912a777fa71e328bea87bc1a95425ca9aaf
Removed peer: b9284ee4c4a27d1d0cb292a9cc2088a430d75bb5b2426430239ec106fd2a6cfc
Node 354f8a05c517336e1929050f3db9e20517757ea252984a2793af897142938c1f stopped.
Closing server socket for Node fbd7416735a39a2abca7227cfec89912a777fa71e328bea87bc1a95425ca9aaf.
Removed peer: 354f8a05c517336e1929050f3db9e20517757ea252984a2793af897142938c1f
Removed peer: b9284ee4c4a27d1d0cb292a9cc2088a430d75bb5b2426430239ec106fd2a6cfc
Node fbd7416735a39a2abca7227cfec89912a777fa71e328bea87bc1a95425ca9aaf stopped.
Closing server socket for Node b9284ee4c4a27d1d0cb292a9cc2088a430d75bb5b2426430239ec106fd2a6cfc.
Removed peer: fbd7416735a39a2abca7227cfec89912a777fa71e328bea87bc1a95425ca9aaf
Removed peer: 354f8a05c517336e1929050f3db9e20517757ea252984a2793af897142938c1f
Node b9284ee4c4a27d1d0cb292a9cc2088a430d75bb5b2426430239ec1